In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.seed(0)
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)
valid = valid.fillna(0)
test = test.fillna(0)

In [9]:
feat_col = ['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location_quick_features', 'audio_properties', 'discrete', 'lf_measurements']
x_train, y_train = train[feat_col], train.iloc[:,-8:]
x_val, y_val = valid[feat_col], valid.iloc[:,-8:]
x_test, y_test = test[feat_col], np.asarray(test.iloc[:,-8:])

In [12]:
latent = 3
def transform_data(data, label):
    transformed_data, transformed_label = [], []
    users = set(data.index.get_level_values(0))
    for u in tqdm(users):
        d = data.loc[u]
        for i in tqdm(range(latent, len(d)), leave=False):
            transformed_data.append(d.iloc[i-latent+1:i+1])
            transformed_label.append(label.loc[u].iloc[i])
    return np.array(transformed_data), np.array(transformed_label)

In [ ]:
x_val, y_val = transform_data(x_val, y_val)
x_test, y_test = transform_data(x_test, y_test)
x_train, y_train = transform_data(x_train, y_train)

In [4]:
scaler = MinMaxScaler() #scale features between 0 and 1
x_train = scaler.fit_transform(x_train)
#x_train = np.asarray(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = scaler.transform(x_test)
#x_test = np.asarray(x_test).reshape(x_test.shape[0], 1, x_test.shape[1])
x_val = scaler.transform(x_val)
#x_val = np.asarray(x_val).reshape(x_val.shape[0], 1, x_val.shape[1])

In [5]:
model = Sequential()
#model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2]), bias_regularizer=L1L2(l1=0.01, l2=0.01)))
model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit([x_train], batch_size=20, y=y_train, verbose=1, validation_data=(x_val, y_val), shuffle=True,
          epochs=1)

Train on 287420 samples, validate on 44632 samples
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[500,2000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node training/Adam/mul_18 (defined at /opt/conda/lib/python3.6/site-packages/keras/optimizers.py:467)  = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Adam/beta_2/read, training/Adam/Variable_16/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics/acc/Mean/_203}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2810_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to Run

In [ ]:
predicted = model.predict(x_test)
predicted = np.argmax(predicted, axis=1)
y_test_decode = np.argmax(y_test, axis=1)
print(accuracy_score(y_test_decode, predicted))

In [ ]:
# evaluate the model
_, acc_train = model.evaluate(x_train, y_train, verbose=0)
_, acc_test = model.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (acc_train, acc_test))
# learning curves of model accuracy
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.legend()
plt.show()